# Данные

In [1]:
import pandas
tweets = pandas.DataFrame.from_csv("Tweets data.csv")

In [2]:
print(tweets[::5000])

        Sentiment                                      SentimentText
ItemID                                                              
1               0                       is so sad for my APL frie...
5002            1       very very very happy and slightly skinny  xx
10012           1  &quot;I feel better now, better than James Bro...
15012           1  ...what can I say ....what a surprise...http:/...
20012           0                                     @ ponyp: hey! 
25012           1       @404world yes I'm going to find one..pronto 
30012           0  @AdamJonesey mmm id love some parma violets ri...
35012           1  @AlTheYid Roast beef and wasabi... yummmmmmmmm...
40012           0  @amandafortier ahhh. yeah. kinda. I tweet from...
45012           0                 @antdeshawn ... the pain is worst 
50012           1  @Ashtarte Just be your usual supportive self. ...
55012           0  @azizabatini86 and that's fine--u kno, the dry...
60012           0       @Bellemord

# preprocessing
Извлечём из твитов признаки

In [5]:
texts = tweets["SentimentText"]
Y = tweets["Sentiment"]

In [13]:
def extract_features(text):
    features = []
    
    
    text = text.lower()
    
    #<для начала будем использовать частоты букв как признаки>
    letters = "abcdefghijklmnopqrstuvwxyz"    
    for l in letters:
        features.append(text.count(l))
    
    return features

In [15]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


# Fit a model
Научим на наших признаках простую линейную модель

In [16]:
#порежем данные
X_train = X[:70000]
Y_train = Y[:70000]
X_test = X[70000:]
Y_test = Y[70000:]

In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
from sklearn.metrics import roc_auc_score
model.coef_

array([[ 0.03976119, -0.02395792,  0.03336063, -0.07673787,  0.01470141,
         0.04481567,  0.04607898,  0.00136932, -0.04444427,  0.14625134,
         0.05731417,  0.04799672, -0.05269558, -0.03541793,  0.01029763,
         0.01853502,  0.23346672,  0.02136664, -0.06216603, -0.03674036,
         0.03427765,  0.02921815, -0.01795578,  0.06048928,  0.04883713,
         0.06188793]])

In [ ]:
Y_pred_train = <предскажите вероятности для обучающей выборки>
Y_pred_test = <предскажите вероятности для обучающей выборки>
print("Обучение:",roc_auc_score(Y_train,Y_pred_train))
print("Тест:",roc_auc_score(Y_test,Y_pred_test))


# opinion mining

Давайте использовуем нашу модель чтобы понять, что людям сейчас нравится/не нравится

In [24]:
import numpy
with open("testdata.txt") as fin:
    test_tweets = fin.read()[:-1].split('\n')
test_tweets = numpy.array(test_tweets)

In [25]:
print test_tweets[:5]

['" I don\'t care what anyone says, I like Hillary Clinton.'
 'have an awesome time at purdue!..'
 "Yep, I'm still in London, which is pretty awesome: P Remind me to post the million and one pictures that I took when I get back to Markham!..."
 "Have to say, I hate Paris Hilton's behavior but I do think she's kinda cute.."
 'i will love the lakers.']


In [26]:
X_ops = []
for tweet in test_tweets:
    X_ops.append(extract_features(tweet))
X_ops = numpy.array(X_ops)


In [28]:
sentiments = model.predict_proba(X_ops)[:,1]
sentiment_order = numpy.argsort(-sentiments)

In [ ]:
### теперь посмотрим на наиболее любимые/ненавистные людям вещи

In [29]:
best_ids = sentiment_order[:10]

for tweet, score in zip(test_tweets[best_ids], sentiments[best_ids]):
    print score,':',tweet

0.999999999999 : OMFG HONDA SUCKS @ @ @ @ @ @ @ @ @ @ @ @ @ FORD A < > L < > L < > THE WAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY.
0.901456613199 : i love sa manthaaaaa aaa aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa aaaaaaaaaaaaaaaa.. < nononeofyouknowherandit'snottheoneyourthinkingabout >..
0.881983628014 : gnun ata talaga pag nageenejoy ka at ngeenjoy ka sa mga kasama mo. you guys make every difficult things in school easy, kc alam k

In [30]:
worst_ids = sentiment_order[-10:]

for tweet, score in zip(test_tweets[worst_ids], sentiments[worst_ids]):
    print score,':',tweet

0.253944425179 : I thought I could be more than just his friend, that the whole friends w / benefits thing wouldn't bother me, but I decided when I was in San Francisco that that is such a horrible idea.
0.253944425179 : I thought I could be more than just his friend, that the whole friends w / benefits thing wouldn't bother me, but I decided when I was in San Francisco that that is such a horrible idea.
0.244636665 : I have two weddings and a birthday party to work at and attend this weekend, and the State Farm inspector is coming on Monday to determine the fate of my beloved Thunderbird.
0.243953203614 : substantiated debt consolidation loans gm mastercard customizing debt elimination He's simply too inept otherwise..
0.231546696305 : the first time, i got to spend time with sansan and tito robbie in the san francisco museum of modern art ( which was awesome, btw.
0.22702564168 : I spent a good bit of time racing around the city on my bike, and last night I went to see Mission Imposs

# bag of words

In [39]:
text_words = [w for ws in text for w in ws.lower().split()]

In [41]:
from collections import Counter
counter = Counter(text_words)

In [40]:
words = list(set(words))
word_to_id = {word:i for i,word in enumerate(list(words))}

NameError: name 'words' is not defined

In [ ]:
#выделите 1000 cлов, которые имеют наиболее разную вероятность войти в позитивный и негативный твит
bag_of_words = <слова-признаки>

In [ ]:
def extract_features(text):
    features = []
    
    words = filter(len,text.lower().split())
    
    <bag of words для слов>
    
    return features

In [ ]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = <обучите модель>

In [ ]:
Y_pred_train = <предскажите вероятности для обучающей выборки copy-paste>
Y_pred_test = <предскажите вероятности для обучающей выборки copy-paste>
print("Обучение:",roc_auc_score(Y_train,Y_pred_train))
print("Тест:",roc_auc_score(Y_test,Y_pred_test))


# collocations
Найдём наиболее частые коллокации и используем их как признаки

In [ ]:
import nltk
from nltk import collocations

In [ ]:
cfinder = collocations.BigramCollocationFinder.from_words(text_words)
cfinder.apply_freq_filter(5)
measure = collocations.BigramAssocMeasures.raw_freq

In [ ]:
print(cfinder.nbest(measure,100))

In [ ]:
def extract_features(text):
    features = []
    
    <bag of words для коллокаций>
    
    return features

In [ ]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = <обучите модель>

In [ ]:
Y_pred_train = <предскажите вероятности для обучающей выборки copy-paste>
Y_pred_test = <предскажите вероятности для обучающей выборки copy-paste>
print("Обучение:",roc_auc_score(Y_train,Y_pred_train))
print("Тест:",roc_auc_score(Y_test,Y_pred_test))
